In [20]:
import pandas as pd 
import numpy as np 
! pip install dask
import dask.dataframe as dd
import dask

from sklearn.model_selection import train_test_split



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
raw = pd.read_csv('./MW-sample.csv',low_memory=False,header= None)

# df = dd.read_csv('./MW.csv')
# df.describe()

FILE FORMAT:

The data is stored in a very simple text format including:

[id]: a numeric, only for reference purposes.

[event] id, a integer, used to distinguish the same event captured at different brain locations, used only by multichannel devices (all except MW).

[device]: a 2 character string, to identify the device used to capture the signals, "MW" for MindWave....

[channel]: a string, to indentify the 10/20 brain location of the signal ....MindWave "FP1"

[code]: a integer, to indentify the digit been thought/seen, with possible values 0,1,2,3,4,5,6,7,8,9 or -1 for random captured signals not related to any of the digits.

[size]: a integer, to identify the size in number of values captured in the 2 seconds of this signal, since the Hz of each device varies, in "theory" the value is close to 512Hz for MW, 128Hz for EP, 220Hz for MU & 128Hz for IN, for each of the 2 seconds.

[data]: a coma separated set of numbers, with the time-series amplitude of the signal, each device uses a different precision to identify the electrical potential captured from the brain: integers in the case of MW & MU or real numbers in the case of EP & IN.

There is no headers in the files,  every line is  a signal, and the fields are separated by a tab

In [3]:
raw.head()

,0,1,2,3,4,5,6,7,8,9,...,942,943,944,945,946,947,948,949,950,951
0,0\t0\tMW\tFP1\t0\t1017\t38,48.0,51.0,44.0,48.0,56.0,56.0,41.0,20.0,-3.0,...,82.0,96.0,92.0,92.0,103.0,118.0,136.0,146.0,145.0,134.0
1,170\t170\tMW\tFP1\t1\t953\t21,19.0,23.0,16.0,1.0,0.0,6.0,9.0,4.0,2.0,...,236.0,242.0,250.0,247.0,235.0,237.0,252.0,275.0,291.0,310.0
2,120\t120\tMW\tFP1\t5\t953\t24,20.0,34.0,54.0,52.0,34.0,19.0,16.0,19.0,28.0,...,290.0,312.0,325.0,328.0,320.0,307.0,298.0,300.0,305.0,300.0
3,180\t180\tMW\tFP1\t0\t1017\t23,32.0,28.0,24.0,21.0,24.0,33.0,36.0,33.0,29.0,...,161.0,163.0,161.0,157.0,145.0,131.0,140.0,161.0,165.0,153.0
4,141\t141\tMW\tFP1\t4\t1017\t6,21.0,26.0,17.0,5.0,4.0,3.0,12.0,25.0,36.0,...,96.0,92.0,107.0,130.0,137.0,120.0,97.0,86.0,99.0,122.0


In [4]:
# https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
tmp = raw[0].str.split('\t').tolist()[:-1] # [:-1] means exclude last
df = pd.DataFrame(tmp,columns = ["id", "event" , "device" , "channel" , "code" , "size" , "0"] )
df.head(3)

,id,event,device,channel,code,size,0
0,0,0,MW,FP1,0,1017,38
1,170,170,MW,FP1,1,953,21
2,120,120,MW,FP1,5,953,24


In [5]:
to_concat = raw[:-1].drop(0,axis=1)
to_concat.head(3)

,1,2,3,4,5,6,7,8,9,10,...,942,943,944,945,946,947,948,949,950,951
0,48.0,51.0,44.0,48.0,56.0,56.0,41.0,20.0,-3.0,-9.0,...,82.0,96.0,92.0,92.0,103.0,118.0,136.0,146.0,145.0,134.0
1,19.0,23.0,16.0,1.0,0.0,6.0,9.0,4.0,2.0,9.0,...,236.0,242.0,250.0,247.0,235.0,237.0,252.0,275.0,291.0,310.0
2,20.0,34.0,54.0,52.0,34.0,19.0,16.0,19.0,28.0,45.0,...,290.0,312.0,325.0,328.0,320.0,307.0,298.0,300.0,305.0,300.0


In [6]:
data = pd.concat([df,to_concat], axis =1)
data = data.drop(['device','channel'],axis =1)
data.head()

,id,event,code,size,0,1,2,3,4,5,...,942,943,944,945,946,947,948,949,950,951
0,0,0,0,1017,38,48.0,51.0,44.0,48.0,56.0,...,82.0,96.0,92.0,92.0,103.0,118.0,136.0,146.0,145.0,134.0
1,170,170,1,953,21,19.0,23.0,16.0,1.0,0.0,...,236.0,242.0,250.0,247.0,235.0,237.0,252.0,275.0,291.0,310.0
2,120,120,5,953,24,20.0,34.0,54.0,52.0,34.0,...,290.0,312.0,325.0,328.0,320.0,307.0,298.0,300.0,305.0,300.0
3,180,180,0,1017,23,32.0,28.0,24.0,21.0,24.0,...,161.0,163.0,161.0,157.0,145.0,131.0,140.0,161.0,165.0,153.0
4,141,141,4,1017,6,21.0,26.0,17.0,5.0,4.0,...,96.0,92.0,107.0,130.0,137.0,120.0,97.0,86.0,99.0,122.0


### Droped 'device' and 'channel', since they are of not in use. this dataset is single channel
... I am using *MindWave*

## Now its time to give a shit about the data we are working with... rather than treating it as 'just data'

[event]: id, a integer, used to distinguish the same event captured at different brain locations, used only by multichannel devices (all except MW).

[size]: a integer, to identify the size in number of values captured in the 2 seconds of this signal, since the Hz of each device varies, in "theory" the value is close to 512Hz for MW, 128Hz for EP, 220Hz for MU & 128Hz for IN, for each of the 2 seconds.

[data]: a coma separated set of numbers, with the time-series amplitude of the signal, each device uses a different precision to identify the electrical potential captured from the brain: integers in the case of MW & MU or real numbers in the case of EP & IN.

## I have truncated the data in excel to make it square (not a *jagged* array) 
* event is of no use
* size appears to be of non use 
* I see no point in keeping ID 

**I have no clue what to do with [data]...**


[data - proper definition]: a time-series signal, where the y-axis is the voltage.... its that fucking simple, no need for big words 

**I'm going back and labeling *data* as 0**

In [7]:
df = data.drop(['id','event','size'],axis =1)
df.head(10)

,code,0,1,2,3,4,5,6,7,8,...,942,943,944,945,946,947,948,949,950,951
0,0,38,48.0,51.0,44.0,48.0,56.0,56.0,41.0,20.0,...,82.0,96.0,92.0,92.0,103.0,118.0,136.0,146.0,145.0,134.0
1,1,21,19.0,23.0,16.0,1.0,0.0,6.0,9.0,4.0,...,236.0,242.0,250.0,247.0,235.0,237.0,252.0,275.0,291.0,310.0
2,5,24,20.0,34.0,54.0,52.0,34.0,19.0,16.0,19.0,...,290.0,312.0,325.0,328.0,320.0,307.0,298.0,300.0,305.0,300.0
3,0,23,32.0,28.0,24.0,21.0,24.0,33.0,36.0,33.0,...,161.0,163.0,161.0,157.0,145.0,131.0,140.0,161.0,165.0,153.0
4,4,6,21.0,26.0,17.0,5.0,4.0,3.0,12.0,25.0,...,96.0,92.0,107.0,130.0,137.0,120.0,97.0,86.0,99.0,122.0
5,0,65,53.0,58.0,74.0,80.0,69.0,56.0,50.0,54.0,...,113.0,132.0,121.0,86.0,82.0,112.0,136.0,141.0,132.0,122.0
6,6,13,37.0,42.0,48.0,51.0,48.0,48.0,51.0,55.0,...,41.0,64.0,77.0,77.0,75.0,80.0,92.0,105.0,108.0,100.0
7,2,12,20.0,21.0,37.0,55.0,58.0,70.0,89.0,85.0,...,131.0,124.0,107.0,112.0,134.0,139.0,124.0,117.0,141.0,161.0
8,4,24,37.0,49.0,55.0,56.0,53.0,57.0,75.0,76.0,...,42.0,75.0,71.0,42.0,26.0,48.0,73.0,72.0,56.0,51.0
9,9,-5,10.0,38.0,48.0,40.0,42.0,58.0,71.0,71.0,...,65.0,64.0,65.0,64.0,52.0,45.0,50.0,61.0,75.0,84.0


In [11]:
X = df.drop(['code'],axis =1)
y = df.code

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [21]:
X_train.head()
y_train.head()

,0,1,2,3,4,5,6,7,8,9,...,942,943,944,945,946,947,948,949,950,951
24,39,26.0,28.0,50.0,65.0,59.0,38.0,9.0,7.0,35.0,...,54.0,44.0,35.0,32.0,25.0,10.0,1.0,6.0,28.0,54.0
154,11,6.0,8.0,17.0,26.0,34.0,37.0,25.0,10.0,3.0,...,8.0,23.0,23.0,13.0,17.0,28.0,38.0,32.0,17.0,18.0
137,2,4.0,8.0,19.0,37.0,53.0,48.0,28.0,22.0,24.0,...,74.0,76.0,73.0,66.0,52.0,44.0,52.0,54.0,48.0,43.0
166,88,86.0,73.0,72.0,76.0,83.0,82.0,84.0,85.0,80.0,...,16.0,18.0,33.0,45.0,48.0,28.0,17.0,12.0,13.0,12.0
128,22,8.0,0.0,1.0,0.0,9.0,35.0,53.0,45.0,35.0,...,109.0,97.0,68.0,59.0,70.0,82.0,81.0,74.0,69.0,72.0


24     2
154    1
137    4
166    9
128    0
Name: code, dtype: object

In [1]:
!pip install tensorflow



In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
tf.test.is_gpu_available()



False

In [6]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [9]:
!pip3 install --upgrade tensorflow-gpu


Requirement already up-to-date: tensorflow-gpu in c:\users\kali\appdata\local\packages\pythonsoftwarefoundation.python.3.7_qbz5n2kfra8p0\localcache\local-packages\python37\site-packages (2.0.0)


In [8]:
tf.test.is_built_with_cuda()

False